# How similar is the text?

In [1]:
import os, sys, spacy

import pandas as pd
import gensim.downloader as api

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../'))

from feature_extraction import SpacyFeatureExtraction
# from data_processing import DataProcessing

In [2]:
load_nlp_model = spacy.load("en_core_web_md")
# pretrained_word_two_vec_model = api.load('word2vec-google-news-300')

In [59]:
mappings = {
    "I predict it will be sunny on 4 April 2025." : ["It was sunny on 4 April 2025.", "It was not sunny on 4 April 2025."],
    "Apple's stock will rise by 10%." : ["Apple's stock rose by 11%.", "Apple's stock rose by 900%.", "Apple's stock rose by 10%."]
}

mappings = {
    "Tesla's stock will rise by 10%." : ["Apple's stock will rise by 10%.", "Apple's stock rose by 11%.", "Apple's stock rose by 10%.", "Apple's stock rose by 900%."]
}

mappings = {
    "Tesla's stock will rise by 10%." : ["Apple's stock will rise by 10%.", "Apple's stock rose by 11%.", "Apple's stock rose by 10%.", "Apple's stock rose by 900%."],
    "Apple's stock will rise by 10%." : ["Apple's stock will rise by 10%.", "Apple's stock rose by 11%.", "Apple's stock rose by 900%.", "Apple's stock rose by 10%."]
}

In [60]:
load_nlp_model = spacy.load("en_core_web_md")

all_mappings = {} # Mapping of predictions to observations
pom_mappings = {} # Mapping of predictions to observations to metrics/scoresl pos can be misleading

oms = []
for prediction in mappings.keys():
    print(f"Prediction: {prediction}")

    om_mappings = {} # Mapping of observations to metrics/scores

    for observation in mappings[prediction]:
        print(f"    Observation: {observation}")

        # Calculate the similarity score
        prediction_doc = load_nlp_model(prediction)
        observation_doc = load_nlp_model(observation)
        similarity = prediction_doc.similarity(observation_doc)
        print(f"    Similarity: {similarity}")
        om_mappings[observation] = [similarity]
        print()   
        # print(f"    OM Mapping: {om_mappings}")
    pom_mappings[prediction] = om_mappings


Prediction: Tesla's stock will rise by 10%.
    Observation: Apple's stock will rise by 10%.
    Similarity: 0.948739442627313

    Observation: Apple's stock rose by 11%.
    Similarity: 0.8743882813030235

    Observation: Apple's stock rose by 10%.
    Similarity: 0.8743882813030235

    Observation: Apple's stock rose by 900%.
    Similarity: 0.8511686894801594

Prediction: Apple's stock will rise by 10%.
    Observation: Apple's stock will rise by 10%.
    Similarity: 1.0

    Observation: Apple's stock rose by 11%.
    Similarity: 0.9399013747850621

    Observation: Apple's stock rose by 900%.
    Similarity: 0.9199637743259186

    Observation: Apple's stock rose by 10%.
    Similarity: 0.9399013747850621



In [61]:
pom_mappings

{"Tesla's stock will rise by 10%.": {"Apple's stock will rise by 10%.": [0.948739442627313],
  "Apple's stock rose by 11%.": [0.8743882813030235],
  "Apple's stock rose by 10%.": [0.8743882813030235],
  "Apple's stock rose by 900%.": [0.8511686894801594]},
 "Apple's stock will rise by 10%.": {"Apple's stock will rise by 10%.": [1.0],
  "Apple's stock rose by 11%.": [0.9399013747850621],
  "Apple's stock rose by 900%.": [0.9199637743259186],
  "Apple's stock rose by 10%.": [0.9399013747850621]}}

In [62]:
# Flatten the dictionary into a list of rows
data = []
for prediction, observations in pom_mappings.items():
    for observation, scores in observations.items():
        data.append([prediction, observation, scores[0]])

# Create the DataFrame
df = pd.DataFrame(data, columns=['Prediction', 'Observation', 'Spacy Similarity'])

# Display the DataFrame
df

,Prediction,Observation,Spacy Similarity
0,Tesla's stock will rise by 10%.,Apple's stock will rise by 10%.,0.948739
1,Tesla's stock will rise by 10%.,Apple's stock rose by 11%.,0.874388
2,Tesla's stock will rise by 10%.,Apple's stock rose by 10%.,0.874388
3,Tesla's stock will rise by 10%.,Apple's stock rose by 900%.,0.851169
4,Apple's stock will rise by 10%.,Apple's stock will rise by 10%.,1.000000
5,Apple's stock will rise by 10%.,Apple's stock rose by 11%.,0.939901
6,Apple's stock will rise by 10%.,Apple's stock rose by 900%.,0.919964
7,Apple's stock will rise by 10%.,Apple's stock rose by 10%.,0.939901


In [67]:
df[0]

KeyError: 0

Comparison
1. No focus on #s, more on words

In [72]:
prediction_doc = load_nlp_model("10%")
observation_doc = load_nlp_model("10.5%")
observation_doc_2 = load_nlp_model("11%")
similarity = prediction_doc.similarity(observation_doc)
similarity

0.7693690410123013

In [73]:
similarity = prediction_doc.similarity(observation_doc_2)
similarity

0.9999999501502246

In [69]:
prediction_doc = load_nlp_model("Apple's stock")
observation_doc = load_nlp_model("Tesla's stock")
similarity = prediction_doc.similarity(observation_doc)
similarity

0.6711231861889055